# Sequence Learning - Direct - English - Testing Session - Boundary Prediction
Version 2: encode all validation set tokens, but keeping only those that are "matching" (= match_status==1)

In [1]:
import torch
import torchaudio
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import plotly.express as px
import random
from sklearn.manifold import TSNE  # Optional: Use t-SNE for dimensionality reduction
import seaborn as sns
from collections import Counter

In [2]:
from padding import generate_mask_from_lengths_mat, mask_it
from paths import *
from my_utils import *
from loss import *
from model import SimplerPhxLearner
from dataset import SeqDatasetBoundary, MelSpecTransform
from my_dataset import DS_Tools
from reshandler import BndEncoderResHandler
from misc_progress_bar import draw_progress_bar
from bnd_detect_tools import *

## Preps

### Dirs

In [3]:
model_save_dir = model_eng_save_dir

log_path = compound_word_log_path
rec_path = word_seg_anno_path

### Constants

In [4]:
EPOCHS = 10
BATCH_SIZE = 1

INPUT_DIM = 64
OUTPUT_DIM = 64

INTER_DIM_0 = 32
INTER_DIM_1 = 16
INTER_DIM_2 = 8

ENC_SIZE_LIST = [INPUT_DIM, INTER_DIM_0, INTER_DIM_1, INTER_DIM_2]
DEC_SIZE_LIST = [OUTPUT_DIM, INTER_DIM_0, INTER_DIM_1, INTER_DIM_2]

DROPOUT = 0.5

REC_SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 64

LOADER_WORKER = 16

## Model

### Model-related defs

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
recon_loss = nn.MSELoss(reduction='none')
masked_recon_loss = MaskedLoss(recon_loss)
model_loss = masked_recon_loss

model = SimplerPhxLearner(enc_size_list=ENC_SIZE_LIST, dec_size_list=DEC_SIZE_LIST, num_layers=2)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

### Load Model

In [6]:
load_ts = "0926172946"
stop_epoch = "194"

In [7]:
model_raw_name = "PT_{}_{}_full".format(load_ts, stop_epoch)
model_name = model_raw_name + ".pt"
model_path = os.path.join(model_save_dir, model_name)
state = torch.load(model_path)

model.load_state_dict(state)
model.to(device)

SimplerPhxLearner(
  (encoder): RLEncoder(
    (rnn): LSTM(64, 16, num_layers=2, batch_first=True)
    (lin_2): LinearPack(
      (linear): Linear(in_features=16, out_features=8, bias=True)
      (relu): Tanh()
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
  (decoder): RALDecoder(
    (rnn): LSTM(64, 8, num_layers=2, batch_first=True)
    (attention): ScaledDotProductAttention(
      (w_q): Linear(in_features=8, out_features=8, bias=True)
      (w_k): Linear(in_features=8, out_features=8, bias=True)
      (w_v): Linear(in_features=8, out_features=8, bias=True)
    )
    (lin_3): LinearPack(
      (linear): Linear(in_features=8, out_features=64, bias=True)
      (relu): Tanh()
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
)

### Dataset

- Note that due to the separate setting of word and phone datasets, we cannot really make it to select those that have not been trained on 
for this test. This is a point to further fix. Make reference to out first work. 

In [8]:
mytrans = MelSpecTransform(sample_rate=REC_SAMPLE_RATE, n_fft=N_FFT, n_mels=N_MELS)
ds = SeqDatasetBoundary(rec_path, os.path.join(log_path, "log.csv"), transform=mytrans)

# Cannot load dataset trained, data filtering applied
valid_ds_indices = DS_Tools.read_indices(os.path.join(model_save_dir, "valid_ds_{}.pkl".format(load_ts)))

valid_ds = torch.utils.data.Subset(ds, valid_ds_indices)

# # this is to reduce the size of the dataset when the training power is not sufficient
# small_len = int(0.5 * len(ds))
# other_len = len(ds) - small_len

# # Randomly split the dataset into train and validation sets
# valid_ds, other_ds = random_split(ds, [small_len, other_len])

valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKER, collate_fn=SeqDatasetBoundary.collate_fn)
valid_num = len(valid_loader.dataset)

In [9]:
valid_num

56808

## Inference

In [10]:
def infer(): 
    model.eval()
    reshandler = BndEncoderResHandler(whole_res_dir=compound_plot_res_path, file_prefix=model_raw_name)
    all_res = []
    all_bnd = []
    all_name = []
    all_ms = []

    total = len(valid_loader)

    for idx, (x, x_lens, bnd, name, ms) in enumerate(valid_loader): 
        bnd = bnd[0]
        name = name[0]
        ms = ms[0]

        x_mask = generate_mask_from_lengths_mat(x_lens, device=device)
        
        x = x.to(device)

        hid_r = model.encode(x, x_lens, x_mask)

        hid_r = hid_r.cpu().detach().numpy().squeeze()

        length = hid_r.shape[0]

        all_res += [hid_r]
        # note that this is bit different, not each frame, but each sequence is treated as one data point
        all_bnd += [bnd]
        all_name += [name]
        all_ms += [ms]

        if idx % 10 == 0: 
            draw_progress_bar(idx, total)
    

    reshandler.res = all_res
    reshandler.tok = all_bnd
    reshandler.name = all_name
    reshandler.match_status = all_ms
    reshandler.save()

In [11]:
if __name__ == "__main__": 
    infer()

[==================================================] 99%	

# Load Res

In [41]:
reshandler = BndEncoderResHandler(whole_res_dir=compound_plot_res_path, file_prefix=model_raw_name)
reshandler.read()

# Evaluate with Wang et al. method

In [42]:
# all
all_preds = reshandler.res
all_bnds = reshandler.tok
all_matching_status = reshandler.match_status

In [43]:
mismatch_counter = 0
total_counter = 0

for ms in all_matching_status: 
    if ms != 1: 
        mismatch_counter += 1
    total_counter += 1

print(mismatch_counter, total_counter, mismatch_counter/total_counter)

17514 56808 0.3083016476552598


In [44]:
# mismatcher_out: exclude those that are not matching
preds_misout = []
bnds_misout = []
for idx in range(len(all_preds)): 
    if all_matching_status[idx] != 1: 
        continue
    preds_misout.append(all_preds[idx])
    bnds_misout.append(all_bnds[idx][:-1])

In [45]:
len(preds_misout)

39294

In [46]:
one_counter = 0
total_counter = 0

for bnd in bnds_misout: 
    if len(bnd) <= 1: 
        one_counter += 1
    total_counter += 1

print(one_counter, total_counter, one_counter/total_counter)

17978 39294 0.4575253219321016


In [47]:
# one_bound_out: bnds excludes last
preds = []
bnds = []
for idx in range(len(preds_misout)): 
    if len(bnds_misout[idx]) <= 1: 
        continue
    preds.append(preds_misout[idx])
    bnds.append(bnds_misout[idx][:-1])

In [48]:
len(bnds)

21316

In [51]:
periodic_preds = periodic_bnd_detect(preds, every=4, only_then_rand=False)

In [52]:
pb_precision_list, pb_recall_list = [], []

#Recall
for i in range(len(bnds)):
    single_utt_recall = tolerance_recall(bnds[i], \
                            periodic_preds[i], tolerance_window=2)
    pb_recall_list.append(single_utt_recall)

#Precision
for i in range(len(periodic_preds)):
    single_utt_precision = tolerance_precision(bnds[i], \
                            periodic_preds[i], tolerance_window=2)
    pb_precision_list.append(single_utt_precision)


precision = sum(pb_precision_list) / len(pb_precision_list)
recall = sum(pb_recall_list) / len(pb_recall_list)
recall *= 100
precision *= 100
if recall == 0. or precision == 0.:
    f_score = -1.
    r_val = -1.
else:
    f_score = (2 * precision * recall) / (precision + recall)
    r_val = r_val_eval(precision, recall)

print("R-value for periodic boundary prediction is: {}".format(r_val))

R-value for periodic boundary prediction is: -29.35992452336693


## Run a through search through all dims

In [60]:
dims = list(range(8)) + ["mean"]
th = [0.1, 0.05, 0.04, 0.035, 0.03]
# th += [0.029, 0.028, 0.027, 0.026, 0.025, 0.024, 0.023, 0.022, 0.021, 0.02, 0.019]
# th += [0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 0.004, 0.003, 0.002, 0.001, 0.0009, 0.0008]
th_total = len(th)

recall_lslsls, precision_lslsls, r_val_lslsls = [], [], []

for idx, dim in enumerate(dims): 
    recall_lsls, precision_lsls, r_val_ls = [], [], []
    
    for idx, thresh in enumerate(th): 
        recall_list, precision_list = thresh_segmentation_eval(batch_delta_hrs(preds, sel=dim, minus=True), bnds, tolerance_window=2, diff_thresh_factor=thresh)
        recall_lsls.append(recall_list)
        precision_lsls.append(precision_list)

    for th_idx in range(len(th)): 
        precision_list = precision_lsls[th_idx]
        recall_list = recall_lsls[th_idx]

        precision = sum(precision_list) / len(precision_list)
        recall = sum(recall_list) / len(recall_list)
        recall *= 100
        precision *= 100
        if recall == 0. or precision == 0.:
            f_score = -1.
            r_val = -1.
        else:
            f_score = (2 * precision * recall) / (precision + recall)
            r_val = r_val_eval(precision, recall)
        
        r_val_ls.append(r_val)
    
    recall_lslsls.append(recall_lsls)
    precision_lslsls.append(precision_lsls)
    r_val_lslsls.append(r_val_ls)
    
    print('The best r_val is: {:.4f}, th = {}, dim = {}'.format(max(r_val_ls), th[r_val_ls.index(max(r_val_ls))], dim))

The best r_val is: 53.4182, th = 0.05, dim = 0
The best r_val is: 47.8377, th = 0.1, dim = 1
The best r_val is: 48.2029, th = 0.04, dim = 2
The best r_val is: 53.5135, th = 0.1, dim = 3
The best r_val is: 29.3523, th = 0.03, dim = 4
The best r_val is: 50.3150, th = 0.05, dim = 5
The best r_val is: 59.8185, th = 0.05, dim = 6
The best r_val is: 51.8937, th = 0.035, dim = 7
The best r_val is: 57.3235, th = 0.03, dim = mean


In [59]:
# Plus
"""
The best r_val is: 64.1704, th = 0.027, dim = 0
The best r_val is: 29.5702, th = 0.0008, dim = 1
The best r_val is: 62.8208, th = 0.019, dim = 2
The best r_val is: 64.6059, th = 0.03, dim = 3
The best r_val is: 49.6899, th = 0.003, dim = 4
The best r_val is: 67.6130, th = 0.008, dim = 5
The best r_val is: 35.5558, th = 0.0008, dim = 6
The best r_val is: 63.5690, th = 0.022, dim = 7
The best r_val is: 55.5981, th = 0.0008, dim = mean
"""

# Minus
"""
The best r_val is: 65.9907, th = 0.024, dim = 0
The best r_val is: 51.9334, th = 0.035, dim = 1
The best r_val is: 62.7616, th = 0.01, dim = 2
The best r_val is: 65.6081, th = 0.03, dim = 3
The best r_val is: 49.1389, th = 0.0008, dim = 4
The best r_val is: 65.7445, th = 0.01, dim = 5
The best r_val is: 65.8716, th = 0.027, dim = 6
The best r_val is: 63.0764, th = 0.019, dim = 7
The best r_val is: 66.7116, th = 0.027, dim = mean
"""


"""
The best r_val is: 64.1667, th = 0.028, dim = 0
The best r_val is: 29.5675, th = 0.001, dim = 1
The best r_val is: 62.7918, th = 0.02, dim = 2
The best r_val is: 64.6059, th = 0.03, dim = 3
The best r_val is: 49.6220, th = 0.004, dim = 4
The best r_val is: 67.6130, th = 0.008, dim = 5
The best r_val is: 35.5199, th = 0.001, dim = 6
The best r_val is: 63.5690, th = 0.022, dim = 7
The best r_val is: 55.5489, th = 0.001, dim = mean
"""

'\nThe best r_val is: 64.1667, th = 0.028, dim = 0\nThe best r_val is: 29.5675, th = 0.001, dim = 1\nThe best r_val is: 62.7918, th = 0.02, dim = 2\nThe best r_val is: 64.6059, th = 0.03, dim = 3\nThe best r_val is: 49.6220, th = 0.004, dim = 4\nThe best r_val is: 67.6130, th = 0.008, dim = 5\nThe best r_val is: 35.5199, th = 0.001, dim = 6\nThe best r_val is: 63.5690, th = 0.022, dim = 7\nThe best r_val is: 55.5489, th = 0.001, dim = mean\n'

# Plot some hidrep graphs in relation to boundary

In [28]:
def plot_feature_with_boundaries(features, boundaries, name):
    num_features = features.shape[1]
    num_samples = features.shape[0]

    time_axis = np.arange(num_samples)

    fig, axes = plt.subplots(num_features + 1, 1, figsize=(10, (num_features + 1) * 2), sharex=True)

    # Plot each feature on separate subplots
    for i in range(num_features):
        ax = axes[i]
        ax.plot(time_axis, features[:, i], label=f'Feature {i + 1}')

        for boundary in boundaries:
            if boundary < num_samples:
                ax.axvline(x=boundary, color='red', linestyle='--', alpha=0.5)

        ax.set_ylabel(f'Feature {i + 1}')

    # Plot all dimensions together with different line colors
    ax = axes[num_features]
    for i in range(num_features):
        ax.plot(time_axis, features[:, i], label=f'Feature {i + 1}', alpha=0.5)

    for boundary in boundaries:
        if boundary < num_samples:
            ax.axvline(x=boundary, color='red', linestyle='--', alpha=0.5)

    ax.set_ylabel('All Features')
    ax.legend()

    plt.xlabel('Time (frames)')

    plt.tight_layout()
    # plt.show()
    plt.savefig("./8dim_hrs/" + "{}_dim.png".format(name))
    plt.close()

In [29]:
def plot_mean_feature_with_boundaries(features, boundaries, name):
    """
    Plot the mean of the features as a line graph with marked boundaries.
    
    Parameters:
    - features: NumPy array of shape (length, 8)
    - boundaries: List of boundary times (frame numbers)
    """
    num_samples = features.shape[0]

    # Create a time axis
    time_axis = np.arange(num_samples)

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(time_axis, features, label='Mean Feature')

    # Mark boundaries on the plot
    for boundary in boundaries:
        if boundary < num_samples:
            plt.axvline(x=boundary, color='red', linestyle='--', alpha=0.5)

    # Set labels and legend
    plt.xlabel('Time (frames)')
    plt.ylabel('Mean Feature Value')
    plt.legend()
    
    # plt.show()
    plt.savefig("./8dim_hrs/" + "{}_mean.png".format(name))
    plt.close()

In [30]:
total_len = len(reshandler.name)

In [31]:
for i in range(20): 
    randidx = random.randint(0, total_len)
    name = reshandler.name[randidx]
    pred = reshandler.res[randidx]
    bnd = reshandler.tok[randidx]

    delta = delta_hrs(pred)
    plot_feature_with_boundaries(delta, bnd, name)

    pred_mean = pred.mean(axis=1)
    delta = delta_hrs(pred_mean)
    plot_mean_feature_with_boundaries(delta, bnd, name)

## Plotter

In [ ]:
def manyOuts2progFrame(manyOuts, groups): 
    df = pd.DataFrame(manyOuts)
    df["name"] = groups

    df = df.sort_values(by=["name"])
    # Group the DataFrame by the grouping column and assign timesteps within each group
    df['timestep'] = df.groupby("name").cumcount() + 1
    return df

# Selective Plot

### Load Res

In [ ]:
reshandler = AnnoEncoderResHandler(whole_res_dir=phone_plot_res_path, file_prefix=model_raw_name)
reshandler.read()

In [ ]:
def select(data, guide, selector):
    # Ensure that the lengths of annotations and data_array match
    if len(guide) != data.shape[0]:
        raise ValueError("The length of guide must match the number of items in the data.")

    # Create a boolean mask for selected annotations
    mask = np.isin(guide, selector)

    # Use the mask to select the corresponding items from the data array
    selected_items = data[mask]

    return selected_items

### Multiplot by Phoneme

In [ ]:
# get usable cluster groups
cluster_groups = ["ng", "hh"]

hidr_cs = select(data=reshandler.res, 
                 guide=reshandler.tok, 
                 selector=cluster_groups)
tags_cs = select(data=np.array(reshandler.tok), 
                 guide=reshandler.tok, 
                 selector=cluster_groups)
color_translate = {item: idx for idx, item in enumerate(cluster_groups)}

# Use Counter to count the occurrences of each item
item_counts = Counter(tags_cs)

# Print the item counts
for item, count in item_counts.items():
    print(f"{item}: {count}")

In [ ]:
X = hidr_cs
y = tags_cs

colors = [color_translate[item] for item in y]
df = pd.DataFrame(X)
df["tag"] = y

sns.set(style="ticks", color_codes=True)
g = sns.pairplot(df,hue='tag', corner=True, diag_kind="kde")
# g.map_lower(sns.kdeplot, levels=4, color=".2")

### Plot by rec

In [ ]:
guide_df = pd.read_csv(os.path.join(phone_plot_res_path, "{}_guide.csv".format(model_raw_name)))
total_len = len(guide_df["Name"].tolist())

all_name = guide_df["Name"]
all_token = guide_df["Token"]

In [ ]:
randidx = random.randint(0, total_len)
save_name = all_name[randidx]
token = all_token[randidx]
print(save_name, token)

In [ ]:
hidr_cs = select(data=reshandler.res, 
                 guide=reshandler.name, 
                 selector=[save_name])

In [ ]:
X = hidr_cs
y = list(range(hidr_cs.shape[0]))

df = pd.DataFrame(X)
df["tag"] = y

sns.set(style="ticks", color_codes=True)
sns.pairplot(df,hue='tag', corner=True)

### Plot by phoneme and rec

In [ ]:
guide_df = pd.read_csv(os.path.join(phone_plot_res_path, "{}_guide.csv".format(model_raw_name)))
filtered_guide_df = guide_df[guide_df['FrameCount'] <= 10]

total_len = len(filtered_guide_df["Name"].tolist())

all_name = filtered_guide_df["Name"].to_numpy()
all_token = filtered_guide_df["Token"]

In [ ]:
target_token = "ng"

names_by_token = select(data=all_name, 
                        guide=all_token, 
                        selector=[target_token])
names_by_token.shape

In [ ]:
data = filtered_guide_df[filtered_guide_df["Token"] == target_token]["FrameCount"].to_numpy().tolist()

# Create a histogram of counts
plt.hist(data,  edgecolor='black')
plt.xlabel('Count')
plt.ylabel('Frequency')
plt.title('Histogram of Counts')
plt.show()

In [ ]:
selnum = 500

In [ ]:
subset_size = selnum if selnum <= names_by_token.shape[0] else names_by_token.shape[0]
random_subset_names_by_token = np.random.choice(names_by_token, size=subset_size, replace=False)

In [ ]:
hidr_cs = select(data=reshandler.res, 
                 guide=reshandler.name, 
                 selector=random_subset_names_by_token)
tags_cs = select(data=np.array(reshandler.name), 
                 guide=reshandler.name, 
                 selector=random_subset_names_by_token)

res_df = manyOuts2progFrame(hidr_cs, tags_cs)
res_df["timestep"].to_numpy().max()

In [ ]:
sns.set(style="ticks", color_codes=True)
sns.pairplot(res_df,hue='timestep', corner=True, palette=sns.color_palette("viridis", res_df["timestep"].to_numpy().max()))